In [5]:
import polars as pl

In [17]:
pitching = pl.read_csv('./Data/lahman_1871-2023_csv/Pitching.csv') 
pitching.head()

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2


In [18]:
batting = pl.read_csv('./Data/lahman_1871-2023_csv/Batting.csv') 
batting.head()

playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""aardsda01""",2004,1,"""SFN""","""NL""",11,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2006,1,"""CHN""","""NL""",45,null,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,null
"""aardsda01""",2007,1,"""CHA""","""AL""",25,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2008,1,"""BOS""","""AL""",47,null,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,null
"""aardsda01""",2009,1,"""SEA""","""AL""",73,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null


In [32]:
ERA = (
    pitching
    .select([pl.col('IPouts'), pl.col('yearID'), pl.col('R'), pl.col('lgID')])
    .filter(pl.col('yearID') >= 1946)
    .group_by(['yearID', 'lgID'])
    .agg([
        pl.col('R').sum().alias('total_runs'),
        pl.col('IPouts').sum().alias('total_ipouts')
    ])
    .with_columns([
        ((pl.col('total_runs') / pl.col('total_ipouts')) * 27).alias('era')
    ])
)
ERA

yearID,lgID,total_runs,total_ipouts,era
i64,str,i64,i64,f64
2014,"""NL""",9709,65445,4.005547
1956,"""AL""",5756,33095,4.695936
1979,"""NL""",8186,52280,4.227659
1961,"""NL""",5600,33080,4.570738
1970,"""AL""",8109,52341,4.183011
…,…,…,…,…
2020,"""AL""",4159,23338,4.811595
1968,"""NL""",5577,44043,3.418909
1981,"""NL""",5035,34906,3.894603
